## Build the structured data files from the raw data:

In [1]:
from fpl_predictor.build_data import DataBuilder

# Run this code to update the structured data:
db = DataBuilder()

# Alternatively run this to access the object without building files:
db = DataBuilder(build_player_files=False, build_master_csv=False)

# Example function:
db.load_year_players(2019).head()

Making master player files ... ✓
Making master player gameweek CSV file ... ✓


,name,code,element_type,first_name,id,second_name,squad_number,team,team_code,web_name
0,Shkodran Mustafi,69140,2,Shkodran,1,Mustafi,None,1,3,Mustafi
1,Héctor Bellerín,98745,2,Héctor,2,Bellerín,None,1,3,Bellerín
2,Sead Kolasinac,111457,2,Sead,3,Kolasinac,None,1,3,Kolasinac
3,Ainsley Maitland-Niles,154043,2,Ainsley,4,Maitland-Niles,None,1,3,Maitland-Niles
4,Sokratis Papastathopoulos,39476,2,Sokratis,5,Papastathopoulos,None,1,3,Sokratis


## Get information about players:

In [2]:
from fpl_predictor.player_information import PlayerInformation
pi = PlayerInformation()
pi.search_players("mo", "salah")

/Users/tp/virtualenvs/fpl-predictor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{118748: 'Mohamed Salah'}

In [3]:
# Get more details from a specific gameweek:
code = 118748
pi.get_player(code, year=2019, week=5)

{'code': 118748,
 'name': 'Mohamed Salah',
 'position': 'MID',
 'value': 125,
 'team': 'Liverpool'}

## Build a Squad:

In [14]:
# Use this object to build a squad:
from fpl_predictor.squad_builder import SquadBuilder

# Scoring metric is the column from the data that you want to rank players by:
sqb = SquadBuilder(scoring_metric="ict_index", n=5)

/Users/tp/virtualenvs/fpl-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  """
/Users/tp/virtualenvs/fpl-predictor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
codes = [38533, 39487, 149484, 206325, 172780, 103955, 118748, 61366, 44699, 
         57127, 193111, 98747, 109533, 80146, 153723]
sqb.squad.remove_all_players()
for code in codes:
    sqb.add_player(code=code, year=2019, week=5)

Player added to squad: Rui Pedro dos Santos Patrício
Player added to squad: Erik Pieters
Player added to squad: Tyrone Mings
Player added to squad: Oleksandr Zinchenko
Player added to squad: James Maddison
Player added to squad: Raheem Sterling
Player added to squad: Mohamed Salah
Player added to squad: Kevin De Bruyne
Player added to squad: Ashley Barnes
Player added to squad: Teemu Pukki
Player added to squad: Todd Cantwell
Player added to squad: Nick Pope
Player added to squad: Emerson Palmieri dos Santos
Player added to squad: Jordan Ayew
Player added to squad: John Lundstram


In [16]:
# Load a squad from CSV:
sqb.squad.remove_all_players()
sqb.squad.load_squad("gw13")

Squad of 15 players loaded from /Users/tp/projects/fpl-predictor/data/squads/gw13.csv


In [17]:
# See the first team chosen by the scoring_metric:
sqb.squad.selected

,code,name,position,value,score,score_per_value,team
0,176413,Christian Pulisic,MID,73,18.6,0.254795,Chelsea
1,103955,Raheem Sterling,MID,123,17.3,0.140650,Man City
2,85971,Heung-Min Son,MID,98,10.2,0.104082,Spurs
3,193111,Todd Cantwell,MID,48,9.2,0.191667,Norwich
4,57127,Teemu Pukki,FWD,70,9.0,0.128571,Norwich
5,173879,Tammy Abraham,FWD,77,8.6,0.111688,Chelsea
6,80146,Jordan Ayew,FWD,51,6.0,0.117647,Crystal Palace
7,171129,Diego Rico,DEF,42,5.8,0.138095,Bournemouth
8,39487,Erik Pieters,DEF,47,3.5,0.074468,Burnley
9,172850,Benjamin Chilwell,DEF,56,3.2,0.057143,Leicester


In [18]:
sqb.squad.substitutes

,code,name,position,value,score,score_per_value,team
0,184341,Mason Mount,MID,67,3.1,0.046269,Chelsea
1,153723,John Lundstram,DEF,42,2.5,0.059524,Sheffield Utd
2,126184,Nathan Aké,DEF,50,1.0,0.020000,Bournemouth
3,38533,Rui Pedro dos Santos Patrício,GK,51,1.0,0.019608,Wolves


In [19]:
sqb.squad.first_team

,code,name,position,score,score_per_value,team,value
0,98747,Nick Pope,GK,3.1,0.067391,Burnley,46.0
1,176413,Christian Pulisic,MID,18.6,0.254795,Chelsea,73.0
2,103955,Raheem Sterling,MID,17.3,0.140650,Man City,123.0
3,85971,Heung-Min Son,MID,10.2,0.104082,Spurs,98.0
4,193111,Todd Cantwell,MID,9.2,0.191667,Norwich,48.0
5,57127,Teemu Pukki,FWD,9.0,0.128571,Norwich,70.0
6,173879,Tammy Abraham,FWD,8.6,0.111688,Chelsea,77.0
7,80146,Jordan Ayew,FWD,6.0,0.117647,Crystal Palace,51.0
8,171129,Diego Rico,DEF,5.8,0.138095,Bournemouth,42.0
9,39487,Erik Pieters,DEF,3.5,0.074468,Burnley,47.0


## Captain & Vice-Captain Picks

In [10]:
sqb.squad.captain

{103955: 'Raheem Sterling'}

In [11]:
sqb.squad.vice_captain

{118748: 'Mohamed Salah'}

## Evaluate potential transfers:

In [16]:
# Evaluate potential transfers from the currently selected squad:
from fpl_predictor.strategies.sample import evaluate_transfers, make_tranfers

# Change the criteria for ranking players:
sqb.cross_seasons = True
sqb.set_n(5)
sqb.set_scoring_metric("ict_index")

# Get suggested transfers:
transfers = evaluate_transfers(sqb, 2019, 13, live=True)
transfers

,available,code,minutes_percent,name,out_code,out_name,out_score,out_score_per_value,out_team,out_value,position,score,score_per_value,team,value,score_gain,score_per_val_gain,val_gain,squad_unavailable,first_team_unavailable
0,1.0,176413.0,0.637778,Christian Pulisic,184341.0,Mason Mount,3.1,0.046269,Chelsea,67.0,MID,13.18,0.178108,Chelsea,74.0,10.08,0.131839,-7.0,False,False
1,1.0,92159.0,0.991111,Jetro Willems,126184.0,Nathan Aké,1.0,0.020000,Bournemouth,50.0,DEF,6.58,0.143043,Newcastle,46.0,5.58,0.123043,4.0,False,False
2,1.0,101188.0,1.000000,Lucas Digne,172850.0,Benjamin Chilwell,3.2,0.057143,Leicester,56.0,DEF,7.42,0.125763,Everton,59.0,4.22,0.068620,-3.0,False,False
3,1.0,92159.0,0.991111,Jetro Willems,153723.0,John Lundstram,2.5,0.059524,Sheffield Utd,42.0,DEF,6.58,0.143043,Newcastle,46.0,4.08,0.083520,-4.0,False,False
4,1.0,92159.0,0.991111,Jetro Willems,39487.0,Erik Pieters,3.5,0.074468,Burnley,47.0,DEF,6.58,0.143043,Newcastle,46.0,3.08,0.068575,1.0,False,False
5,1.0,176413.0,0.637778,Christian Pulisic,85971.0,Heung-Min Son,10.2,0.104082,Spurs,98.0,MID,13.18,0.178108,Chelsea,74.0,2.98,0.074026,24.0,False,False
6,1.0,40694.0,1.000000,Roberto Jimenez Gago,38533.0,Rui Pedro dos Santos Patrício,1.0,0.019608,Wolves,51.0,GK,3.48,0.079091,West Ham,44.0,2.48,0.059483,7.0,False,False
7,1.0,102057.0,0.997778,Raúl Jiménez,173879.0,Tammy Abraham,8.6,0.111688,Chelsea,77.0,FWD,10.56,0.142703,Wolves,74.0,1.96,0.031014,3.0,False,False
8,1.0,102057.0,0.997778,Raúl Jiménez,57127.0,Teemu Pukki,9.0,0.128571,Norwich,70.0,FWD,10.56,0.142703,Wolves,74.0,1.56,0.014131,-4.0,False,False
9,1.0,178304.0,0.580000,Lys Mousset,80146.0,Jordan Ayew,6.0,0.117647,Crystal Palace,51.0,FWD,6.92,0.138400,Sheffield Utd,50.0,0.92,0.020753,1.0,False,False


## Make Transfers:

In [18]:
sqb.squad.remove_player(118748)

Removed player from squad (14 remain): Mohamed Salah


In [20]:
sqb.add_player(code=176413.0, year=2019, week=12, live=False)

Player added to squad: Christian Pulisic


In [21]:
# Save a squad to CSV:
sqb.squad.save_squad("gw13")

Squad of 15 players saved at: /Users/tp/projects/fpl-predictor/data/squads/gw13.csv
